<a href="https://colab.research.google.com/github/aakashrana7/LangChain/blob/main/Langchain_Class2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Few-Shot Prompting (LangChain)

Meaning:
Few-shot prompting is when you give an LLM a few examples of the input and output format inside the prompt so it learns from them and generates answers in the same style.
It’s like saying: “Here are some examples. Now do the same for this new case.”

Few-Shot Prompt Template in LangChain

In LangChain, you can use FewShotPromptTemplate to automatically insert multiple examples into a single prompt.

Main parameters:

examples → A list of dictionaries with your example input/output.

example_prompt → A PromptTemplate that formats how each example looks.

prefix → Text that comes before the examples (e.g., instructions).

suffix → Text that comes after the examples (usually the real question).

input_variables → Variables in the suffix that will be filled with user input.

In [ ]:
from langchain.prompts import PromptTemplate, FewShotPromptTemplate

# 1. Define how each example looks
example_prompt = PromptTemplate(
    input_variables=["question", "answer"],
    template="Q: {question}\nA: {answer}"
)

# 2. Provide few-shot examples
examples = [
    {"question": "Capital of France?", "answer": "Paris"},
    {"question": "Capital of Japan?", "answer": "Tokyo"},
]

# 3. Create the FewShotPromptTemplate
few_shot = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix="Answer the following questions:",
    suffix="Q: {user_question}\nA:",
    input_variables=["user_question"]
)

# 4. Format the final prompt
print(few_shot.format(user_question="Capital of Nepal?"))


Building prompts for sequential chains


In [ ]:
# Create a prompt template that takes an input activity
learning_prompt = PromptTemplate(
    input_variables=["activity"],
    template="I want to learn how to {activity}. Can you suggest how I can learn this step-by-step?"
)

# Create a prompt template that places a time constraint on the output
time_prompt = PromptTemplate(
    input_variables=["learning_plan"],
    template="I only have one week. Can you create a plan to help me hit this goal: {learning_plan}."
)

# Invoke the learning_prompt with an activity
print(learning_prompt.invoke({"activity": "play_golf"}))

Sequential chains with LCEL


In [ ]:
learning_prompt = PromptTemplate(
    input_variables=["activity"],
    template="I want to learn how to {activity}. Can you suggest how I can learn this step-by-step?"
)

time_prompt = PromptTemplate(
    input_variables=["learning_plan"],
    template="I only have one week. Can you create a concise plan to help me hit this goal: {learning_plan}."
)

# Complete the sequential chain with LCEL
seq_chain = ({"learning_plan": learning_prompt | llm | StrOutputParser()}
    | time_prompt
    | llm
    | StrOutputParser())

# Call the chain
print(seq_chain.invoke({"activity": "play the harmonica"}))

#RAG


1. Document Loaders

In [ ]:
!pip install pypdf unstructured


In [ ]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("path/to/file/attention_is_all_you_need.pdf")
data = loader.load()
print(data[0])


In [ ]:
from langchain_community.document_loaders.csv_loader import CSVLoader

loader = CSVLoader('fifa_countries_audience.csv')
data = loader.load()
print(data[0])


In [ ]:
from langchain_community.document_loaders import Unstructured

HTMLLoaderloader = UnstructuredHTMLLoader("white_house_executive_order_nov_2023.html")

data = loader.load()
print(data[0])
print(data[0].metadata)

2. Splitting external data for retrieval



>using CharacterTextSplitter for single separator



In [ ]:
# Import the character splitter
from langchain_text_splitters import CharacterTextSplitter

quote = 'Words are flowing out like endless rain into a paper cup,\nthey slither while they pass,\nthey slip away across the universe.'
chunk_size = 24
chunk_overlap = 10

# Create an instance of the splitter class
splitter = CharacterTextSplitter(
    separator='\n',
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap
)

# Split the string and print the chunks
docs = splitter.split_text(quote)
print(docs)
print([len(doc) for doc in docs])

>using RecursiveCharacterTextSplitter for multiple separator left to right

In [ ]:
# Import the recursive character splitter
from langchain_text_splitters import RecursiveCharacterTextSplitter

quote = 'Words are flowing out like endless rain into a paper cup,\nthey slither while they pass,\nthey slip away across the universe.'
chunk_size = 24
chunk_overlap = 10

# Create an instance of the splitter class
splitter = RecursiveCharacterTextSplitter(
    separators=["\n\n", "\n", " ", ""],
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap
)

# Split the document and print the chunks
docs = splitter.split_text(quote)
print(docs)
print([len(doc) for doc in docs])

In [ ]:
# Load the HTML document into memory
loader = UnstructuredHTMLLoader('white_house_executive_order_nov_2023.html')
data = loader.load()

# Define variables
chunk_size = 300
chunk_overlap = 100

# Split the HTML
splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap,
    separators=['.'])

docs = splitter.split_documents(data)
print(docs)

3. Storage and Retrieval

In [ ]:
loader = PyPDFLoader('rag_vs_fine_tuning.pdf')
data = loader.load()

# Split the document using RecursiveCharacterTextSplitter
splitter = RecursiveCharacterTextSplitter(
    chunk_size=300,
    chunk_overlap=50)
docs = splitter.split_documents(data)

# Embed the documents in a persistent Chroma vector database
embedding_function = OpenAIEmbeddings(api_key='<OPENAI_API_TOKEN>', model='text-embedding-3-small')
vectorstore = Chroma.from_documents(
    docs,
    embedding=embedding_function,
    persist_directory=os.getcwd()
)

# Configure the vector store as a retriever
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 3}
)

In [ ]:
vectorstore = Chroma.from_documents(
    docs,
    embedding=OpenAIEmbeddings(api_key='<OPENAI_API_TOKEN>', model='text-embedding-3-small'),
    persist_directory=os.getcwd()
)

retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 3}
)

# Create a chain to link retriever, prompt_template, and llm
rag_chain = ({"context": retriever, "question": RunnablePassthrough()}
            | prompt_template
            | llm)

# Invoke the chain
response = rag_chain.invoke("Which popular LLMs were considered in the paper?")
print(response.content)